### Configuration

In [0]:
### google drive mount ###
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
### configuration ###
# import numpy and pandas
import numpy as np
import pandas as pd

# used for dates
import datetime as dt

# for collecting open api data
import urllib.request
import json
from pandas.io.json import json_normalize

# config
DATA_PATH = "/content/gdrive/My Drive/data/" 
# local 실행시
# DATA_PATH = "../data/"  #코드파일의 위치는 ~/code/ data폴더의 위치는 ~/code/data/
ENCODING = 'CP949'

In [0]:
### collect & concat data function ###

def make_path(add_path):
    path_list = [DATA_PATH]
    path_list.append(add_path)
    return ''.join(path_list)

def read_file(path, encode=ENCODING):
    df = pd.read_csv(path, encoding=encode, dtype=str)
    return df

def read_files(path, encode=ENCODING):
    df_from_each_file = (
        pd.read_csv(f, encoding=encode, dtype=str) for f in path
        )
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    return concatenated_df

### 1)  Data Preprocessing

### 1-1. 승하차인원 Data Preprocessing

In [0]:
df = read_file(make_path("승하차인원.csv"))
df

,날짜,호선,역번호,역명,구분,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,13 ~ 14,14 ~ 15,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후,합 계
0,2019-01-01,1호선,150,서울역,승차,348,321,348,741,940,1401,2166,2844,3055,2341,3370,3226,3447,3277,2898,2590,3004,2234,970,38,39559
1,2019-01-01,1호선,150,서울역,하차,222,821,808,961,1472,1737,1917,2316,2539,2360,2743,2491,2572,2060,1991,1683,1217,764,456,149,31279
2,2019-01-01,1호선,151,시청,승차,87,98,143,206,280,395,500,672,673,868,946,958,1062,1146,1119,992,904,570,211,6,11836
3,2019-01-01,1호선,151,시청,하차,48,237,323,571,577,648,744,960,996,875,964,733,746,606,374,311,268,177,144,57,10359
4,2019-01-01,1호선,152,종각,승차,669,318,217,203,333,365,666,802,1103,1294,1843,1870,2129,2024,1806,1796,1780,1296,497,31,21042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200799,2019-12-31,8호선,2825,신흥,하차,20,87,86,243,208,189,205,327,347,381,382,501,510,562,493,378,301,257,154,163,5794
200800,2019-12-31,8호선,2826,수진,승차,111,224,542,813,436,343,267,350,297,285,407,427,436,395,245,219,155,126,68,82,6228
200801,2019-12-31,8호선,2826,수진,하차,27,97,115,433,259,198,207,283,381,376,404,373,463,501,452,354,332,322,256,163,5996
200802,2019-12-31,8호선,2827,모란,승차,73,95,342,482,253,210,253,281,377,350,303,366,375,418,242,199,152,189,126,65,5151


In [0]:
values=list(df.columns[5:-1])
id=[item for item in df.columns if item not in values]
df=pd.melt(df,id_vars=id,value_vars=values)
df['구분']=df['구분'].replace('승차',1)
df['구분']=df['구분'].replace('하차',0)
df

,날짜,호선,역번호,역명,구분,합 계,variable,value
0,2019-01-01,1호선,150,서울역,1,39559,06시 이전,348
1,2019-01-01,1호선,150,서울역,0,31279,06시 이전,222
2,2019-01-01,1호선,151,시청,1,11836,06시 이전,87
3,2019-01-01,1호선,151,시청,0,10359,06시 이전,48
4,2019-01-01,1호선,152,종각,1,21042,06시 이전,669
...,...,...,...,...,...,...,...,...
4016075,2019-12-31,8호선,2825,신흥,0,5794,24시 이후,163
4016076,2019-12-31,8호선,2826,수진,1,6228,24시 이후,82
4016077,2019-12-31,8호선,2826,수진,0,5996,24시 이후,163
4016078,2019-12-31,8호선,2827,모란,1,5151,24시 이후,65


In [0]:
df['date']=pd.to_datetime(df['날짜'])
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df.호선 = df.호선.str.extract(r'([\d]+)',expand=True)
df['호선'] = pd.to_numeric(df['호선'])
onoff = df.loc[df.호선 < 5, :]
onoff

,날짜,호선,역번호,역명,구분,합 계,variable,value,date,year,month,day
0,2019-01-01,1,150,서울역,1,39559,06시 이전,348,2019-01-01,2019,1,1
1,2019-01-01,1,150,서울역,0,31279,06시 이전,222,2019-01-01,2019,1,1
2,2019-01-01,1,151,시청,1,11836,06시 이전,87,2019-01-01,2019,1,1
3,2019-01-01,1,151,시청,0,10359,06시 이전,48,2019-01-01,2019,1,1
4,2019-01-01,1,152,종각,1,21042,06시 이전,669,2019-01-01,2019,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4015763,2019-12-31,4,432,총신대입구(이수),0,24185,24시 이후,513,2019-12-31,2019,12,31
4015764,2019-12-31,4,433,사당,1,29318,24시 이후,178,2019-12-31,2019,12,31
4015765,2019-12-31,4,433,사당,0,24573,24시 이후,934,2019-12-31,2019,12,31
4015766,2019-12-31,4,434,남태령,1,1455,24시 이후,5,2019-12-31,2019,12,31


### 1-2. 승하차인원 + 역주소 Preprocessing

In [0]:
# 역주소 가져오기
df = pd.read_csv(make_path("addr.csv"), encoding = ENCODING, 
                 dtype=str, usecols = ['STATION_CD', 'STATION_NM', 'ADDRESS'])
df

,STATION_CD,STATION_NM,ADDRESS
0,0150,서울,서울특별시 중구 세종대로 지하 2
1,1762,오이도,경기도 시흥시 역전로 430
2,0312,불광,서울특별시 은평구 통일로 지하 723-1
3,0339,수서,서울특별시 강남구 광평로 지하 270
4,1803,역곡,경기도 부천시 원미구 역곡로 1
...,...,...,...
235,0417,길음,서울특별시 성북구 동소문로 지하 248
236,1021,녹천,서울특별시 도봉구 덕릉로 376
237,0313,녹번,서울특별시 은평구 통일로 지하 602-1
238,1705,관악,경기도 안양시 만안구 경수대로 1273번길 46


In [0]:
# 역주소에서 구 뽑아내기
df = df.loc[df['ADDRESS'].str.contains('서울', na=False)]
df['구'] = df['ADDRESS'].str.split(' ').str[1]
ad = df[['STATION_CD', '구']]
ad.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,STATION_CD,구
0,0150,중구
2,0312,은평구
3,0339,강남구


In [0]:
ad.dtypes

STATION_CD    object
구             object
dtype: object

In [0]:
# 역번호 기준으로 승하차인원 + 역주소 merge
onoff['역번호'] = onoff['역번호'].str.zfill(4)
ad.columns = ['역번호', '구']
merge1 = pd.merge(onoff, ad, how='left', on='역번호')
merge1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,날짜,호선,역번호,역명,구분,합 계,variable,value,date,year,month,day,구
0,2019-01-01,1,0150,서울역,1,39559,06시 이전,348,2019-01-01,2019,1,1,중구
1,2019-01-01,1,0150,서울역,0,31279,06시 이전,222,2019-01-01,2019,1,1,중구
2,2019-01-01,1,0151,시청,1,11836,06시 이전,87,2019-01-01,2019,1,1,중구
3,2019-01-01,1,0151,시청,0,10359,06시 이전,48,2019-01-01,2019,1,1,중구
4,2019-01-01,1,0152,종각,1,21042,06시 이전,669,2019-01-01,2019,1,1,종로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738155,2019-12-31,4,0432,총신대입구(이수),0,24185,24시 이후,513,2019-12-31,2019,12,31,동작구
1738156,2019-12-31,4,0433,사당,1,29318,24시 이후,178,2019-12-31,2019,12,31,동작구
1738157,2019-12-31,4,0433,사당,0,24573,24시 이후,934,2019-12-31,2019,12,31,동작구
1738158,2019-12-31,4,0434,남태령,1,1455,24시 이후,5,2019-12-31,2019,12,31,서초구


In [0]:
merge1.drop('합 계', axis = 1, inplace = True)
merge1.head(1)

,날짜,호선,역번호,역명,구분,variable,value,date,year,month,day,구
0,2019-01-01,1,0150,서울역,1,06시 이전,348,2019-01-01,2019,1,1,중구


In [0]:
merge1.columns=['날짜','호선','역번호','역명','승하차','시간대','승하차인원','date','year','month','day','구']
merge1.head(1)

,날짜,호선,역번호,역명,승하차,시간대,승하차인원,date,year,month,day,구
0,2019-01-01,1,0150,서울역,1,06시 이전,348,2019-01-01,2019,1,1,중구


In [0]:
merge1.시간대.unique()

array(['06시 이전', '06 ~ 07', '07 ~ 08', '08 ~ 09', '09 ~ 10', '10 ~ 11',
       '11 ~ 12', '12 ~ 13', '13 ~ 14', '14 ~ 15', '15 ~ 16', '16 ~ 17',
       '17 ~ 18', '18 ~ 19', '19 ~ 20', '20 ~ 21', '21 ~ 22', '22 ~ 23',
       '23 ~ 24', '24시 이후'], dtype=object)

In [0]:
# 시간대 06~07 형태에서 06 형태로 변환
merge1['시간대'] = merge1['시간대'].str.split(" ").str[0]
merge1.시간대.unique()

array(['06시', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24시'],
      dtype=object)

In [0]:
merge1['시간대'] = merge1['시간대'].apply(lambda x: 
                                            'BEFORE_06' if x =="06시"
                                            else 'AFTER_24' if x == "24시"
                                            else x)
merge1.시간대.unique()

array(['BEFORE_06', '06', '07', '08', '09', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', 'AFTER_24'],
      dtype=object)

In [0]:
merge1 = merge1.loc[(merge1.시간대 != 'BEFORE_06') & (merge1.시간대 != 'AFTER_24')]
merge1.시간대.unique()

array(['06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'], dtype=object)

In [0]:
merge1['승하차인원']=merge1.승하차인원.astype(int)
merge1.dtypes

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


날짜               object
호선                int64
역번호              object
역명               object
승하차               int64
시간대              object
승하차인원             int64
date     datetime64[ns]
year              int64
month             int64
day               int64
구                object
dtype: object

In [0]:
merge1 =merge1.groupby(["날짜", "호선", "역번호", "역명", "시간대", "date", "year", "month", "day", "구"], as_index=False)["승하차인원"].sum()
merge1

,날짜,호선,역번호,역명,시간대,date,year,month,day,구,승하차인원
0,2019-01-01,1,0150,서울역,06,2019-01-01,2019,1,1,중구,1142
1,2019-01-01,1,0150,서울역,07,2019-01-01,2019,1,1,중구,1156
2,2019-01-01,1,0150,서울역,08,2019-01-01,2019,1,1,중구,1702
3,2019-01-01,1,0150,서울역,09,2019-01-01,2019,1,1,중구,2412
4,2019-01-01,1,0150,서울역,10,2019-01-01,2019,1,1,중구,3138
...,...,...,...,...,...,...,...,...,...,...,...
775597,2019-12-31,4,0434,남태령,19,2019-12-31,2019,12,31,서초구,127
775598,2019-12-31,4,0434,남태령,20,2019-12-31,2019,12,31,서초구,84
775599,2019-12-31,4,0434,남태령,21,2019-12-31,2019,12,31,서초구,85
775600,2019-12-31,4,0434,남태령,22,2019-12-31,2019,12,31,서초구,66


### 1-3. 승하차인원 + 유동인구수 Preprocessing


In [0]:
# 유동인구수 가져오기
pop = pd.read_csv(make_path("FLT_SEOUL_10MONTH.csv"), dtype=str)
pop['유동인구수'] = pop['유동인구수'].astype(int)
pop['시간대'] = pop['시간(1시간단위)'].astype(int)
pop

,일자,시간(1시간단위),연령대(10세단위),성별,시,군구,유동인구수,시간대
0,20191001,00,20,남성,서울,광진구,32760,0
1,20191001,00,20,남성,서울,마포구,30480,0
2,20191001,00,20,남성,서울,서초구,27250,0
3,20191001,00,20,여성,서울,관악구,49870,0
4,20191001,00,30,남성,서울,강서구,51420,0
...,...,...,...,...,...,...,...,...
215695,20191031,23,60,여성,서울,관악구,28330,23
215696,20191031,23,70,남성,서울,강서구,16270,23
215697,20191031,23,70,남성,서울,용산구,8470,23
215698,20191031,23,70,여성,서울,강서구,23940,23


In [0]:
pop.시간대.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [0]:
# 시간대 변환
pop['시간대'] = pop['시간대'].apply(lambda x: 
                                            'BEFORE_06' if ( x > 0 and x < 6 )
                                            else 'AFTER_24' if x == 0
                                            else str(x).zfill(2))
pop.시간대.unique()

array(['AFTER_24', 'BEFORE_06', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
      dtype=object)

In [0]:
pop = pop.loc[(pop.시간대 != 'BEFORE_06') & (pop.시간대 != 'AFTER_24')]
pop.시간대.unique()

array(['06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'], dtype=object)

In [0]:
pop_gr = pop.groupby(['일자','시간대', '군구'], as_index=False)['유동인구수'].sum()
pop_gr

,일자,시간대,군구,유동인구수
0,20191001,06,강남구,484890
1,20191001,06,강동구,345450
2,20191001,06,강북구,240870
3,20191001,06,강서구,453730
4,20191001,06,관악구,420730
...,...,...,...,...
13470,20191031,23,용산구,216160
13471,20191031,23,은평구,365290
13472,20191031,23,종로구,155290
13473,20191031,23,중구,152320


In [0]:
pop_gr.rename(columns={'군구' : '구'}, inplace=True)
pop_gr.head(2)

,일자,시간대,구,유동인구수
0,20191001,06,강남구,484890
1,20191001,06,강동구,345450


In [0]:
# 조인할 승하차인원+역주소 가져오기 (2019년 10월분)
merge1 = merge1.loc[merge1.month==10]
merge1

,날짜,호선,역번호,역명,시간대,date,year,month,day,구,승하차인원
579852,2019-10-01,1,0150,서울역,06,2019-10-01,2019,10,1,중구,2446
579853,2019-10-01,1,0150,서울역,07,2019-10-01,2019,10,1,중구,6448
579854,2019-10-01,1,0150,서울역,08,2019-10-01,2019,10,1,중구,13578
579855,2019-10-01,1,0150,서울역,09,2019-10-01,2019,10,1,중구,9203
579856,2019-10-01,1,0150,서울역,10,2019-10-01,2019,10,1,중구,4982
...,...,...,...,...,...,...,...,...,...,...,...
645691,2019-10-31,4,0434,남태령,19,2019-10-31,2019,10,31,서초구,147
645692,2019-10-31,4,0434,남태령,20,2019-10-31,2019,10,31,서초구,110
645693,2019-10-31,4,0434,남태령,21,2019-10-31,2019,10,31,서초구,85
645694,2019-10-31,4,0434,남태령,22,2019-10-31,2019,10,31,서초구,70


In [0]:
merge1['일자'] = merge1['date'].apply(lambda x: x.strftime('%Y%m%d'))
merge1['일자']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


579852    20191001
579853    20191001
579854    20191001
579855    20191001
579856    20191001
            ...   
645691    20191031
645692    20191031
645693    20191031
645694    20191031
645695    20191031
Name: 일자, Length: 65844, dtype: object

In [0]:
pop_gr.일자.unique()

array(['20191001', '20191002', '20191003', '20191004', '20191005',
       '20191006', '20191007', '20191008', '20191009', '20191010',
       '20191011', '20191012', '20191013', '20191014', '20191015',
       '20191016', '20191017', '20191018', '20191020', '20191021',
       '20191022', '20191023', '20191024', '20191025', '20191026',
       '20191027', '20191028', '20191029', '20191030', '20191031'],
      dtype=object)

In [0]:
merge1.일자.unique()

array(['20191001', '20191002', '20191003', '20191004', '20191005',
       '20191006', '20191007', '20191008', '20191009', '20191010',
       '20191011', '20191012', '20191013', '20191014', '20191015',
       '20191016', '20191017', '20191018', '20191019', '20191020',
       '20191021', '20191022', '20191023', '20191024', '20191025',
       '20191026', '20191027', '20191028', '20191029', '20191030',
       '20191031'], dtype=object)

In [0]:
# 승하차인원+역주소+유동인구수 merge
merge2 = pd.merge(merge1, pop_gr, how='left', on=['일자', '시간대', '구'])
merge2

,날짜,호선,역번호,역명,시간대,date,year,month,day,구,승하차인원,일자,유동인구수
0,2019-10-01,1,0150,서울역,06,2019-10-01,2019,10,1,중구,2446,20191001,153060.0
1,2019-10-01,1,0150,서울역,07,2019-10-01,2019,10,1,중구,6448,20191001,231170.0
2,2019-10-01,1,0150,서울역,08,2019-10-01,2019,10,1,중구,13578,20191001,370810.0
3,2019-10-01,1,0150,서울역,09,2019-10-01,2019,10,1,중구,9203,20191001,428280.0
4,2019-10-01,1,0150,서울역,10,2019-10-01,2019,10,1,중구,4982,20191001,453590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65839,2019-10-31,4,0434,남태령,19,2019-10-31,2019,10,31,서초구,147,20191031,464040.0
65840,2019-10-31,4,0434,남태령,20,2019-10-31,2019,10,31,서초구,110,20191031,433020.0
65841,2019-10-31,4,0434,남태령,21,2019-10-31,2019,10,31,서초구,85,20191031,409520.0
65842,2019-10-31,4,0434,남태령,22,2019-10-31,2019,10,31,서초구,70,20191031,385540.0


In [0]:
merge2.시간대.unique()

array(['06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'], dtype=object)

In [0]:
merge2.rename(columns={'날짜' : 'BAS_DT'}, inplace=True)

In [0]:
f = merge2[(merge2.BAS_DT != '2019-10-06') & (merge2.BAS_DT != '2019-10-13') & (merge2.BAS_DT != '2019-10-20') & (merge2.BAS_DT != '2019-10-27') & (merge2.BAS_DT != '2019-10-03') & (merge2.BAS_DT != '2019-10-09')]
f.BAS_DT.unique()

array(['2019-10-01', '2019-10-02', '2019-10-04', '2019-10-05',
       '2019-10-07', '2019-10-08', '2019-10-10', '2019-10-11',
       '2019-10-12', '2019-10-14', '2019-10-15', '2019-10-16',
       '2019-10-17', '2019-10-18', '2019-10-19', '2019-10-21',
       '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25',
       '2019-10-26', '2019-10-28', '2019-10-29', '2019-10-30',
       '2019-10-31'], dtype=object)

In [0]:
f.rename(columns={'BAS_DT' :'날짜' }, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### 2) 환승역 여부 표시 

In [0]:
df = f
df

,날짜,호선,역번호,역명,시간대,date,year,month,day,구,승하차인원,일자,유동인구수
0,2019-10-01,1,0150,서울역,06,2019-10-01,2019,10,1,중구,2446,20191001,153060.0
1,2019-10-01,1,0150,서울역,07,2019-10-01,2019,10,1,중구,6448,20191001,231170.0
2,2019-10-01,1,0150,서울역,08,2019-10-01,2019,10,1,중구,13578,20191001,370810.0
3,2019-10-01,1,0150,서울역,09,2019-10-01,2019,10,1,중구,9203,20191001,428280.0
4,2019-10-01,1,0150,서울역,10,2019-10-01,2019,10,1,중구,4982,20191001,453590.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65839,2019-10-31,4,0434,남태령,19,2019-10-31,2019,10,31,서초구,147,20191031,464040.0
65840,2019-10-31,4,0434,남태령,20,2019-10-31,2019,10,31,서초구,110,20191031,433020.0
65841,2019-10-31,4,0434,남태령,21,2019-10-31,2019,10,31,서초구,85,20191031,409520.0
65842,2019-10-31,4,0434,남태령,22,2019-10-31,2019,10,31,서초구,70,20191031,385540.0


In [0]:
df['TRANSFER'] = df['역번호'].replace(['150','151','153','155','156','158','159','201','203','204','205','206','208','211','212','216','218','220','222','223','226','233','234','236','237','238','239','243','311','312','319','320','323','329','330','332','334','339','340','342','411','412','418','421','422','423','426','428','430','431','432','433'],
                                   ['1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1'],regex = True)

df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,날짜,호선,역번호,역명,시간대,date,year,month,day,구,승하차인원,일자,유동인구수,TRANSFER
0,2019-10-01,1,0150,서울역,06,2019-10-01,2019,10,1,중구,2446,20191001,153060.0,01
1,2019-10-01,1,0150,서울역,07,2019-10-01,2019,10,1,중구,6448,20191001,231170.0,01
2,2019-10-01,1,0150,서울역,08,2019-10-01,2019,10,1,중구,13578,20191001,370810.0,01
3,2019-10-01,1,0150,서울역,09,2019-10-01,2019,10,1,중구,9203,20191001,428280.0,01
4,2019-10-01,1,0150,서울역,10,2019-10-01,2019,10,1,중구,4982,20191001,453590.0,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65839,2019-10-31,4,0434,남태령,19,2019-10-31,2019,10,31,서초구,147,20191031,464040.0,0434
65840,2019-10-31,4,0434,남태령,20,2019-10-31,2019,10,31,서초구,110,20191031,433020.0,0434
65841,2019-10-31,4,0434,남태령,21,2019-10-31,2019,10,31,서초구,85,20191031,409520.0,0434
65842,2019-10-31,4,0434,남태령,22,2019-10-31,2019,10,31,서초구,70,20191031,385540.0,0434


In [0]:
df['TRANSFER'] = df['TRANSFER'].apply(lambda x: 
                                            '0' if x != '01'
                                            else x)
df.TRANSFER.unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['01', '0'], dtype=object)

In [0]:
df.TRANSFER = df.TRANSFER.astype(int)
df.TRANSFER.unique()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


array([1, 0])

In [0]:
merge2 = df

In [0]:
merge2.isnull().sum()

날짜             0
호선             0
역번호            0
역명             0
시간대            0
date           0
year           0
month          0
day            0
구              0
승하차인원          0
일자             0
유동인구수       2124
TRANSFER       0
dtype: int64

In [0]:
# 승하차인원+유동인구 csv로 저장 -> data_stats에서 사용
merge2.to_csv(make_path("유동인구.csv"), encoding='cp949', index=False)

### 3 . Accuracy Check용 혼잡도 Data Preprocessing

In [0]:
df = read_file(make_path("서울교통공사_혼잡도_2017.csv"))
df

,사용일,호선,구분,역명,역번,05:30 ~,06:00 ~,06:30 ~,07:00 ~,07:30 ~,08:00 ~,08:30 ~,09:00 ~,09:30 ~,10:00 ~,10:30 ~,11:00 ~,11:30 ~,12:00 ~,12:30 ~,13:00 ~,13:30 ~,14:00 ~,14:30 ~,15:00 ~,15:30 ~,16:00 ~,16:30 ~,17:00 ~,17:30 ~,18:00 ~,18:30 ~,19:00 ~,19:30 ~,20:00 ~,20:30 ~,21:00 ~,21:30 ~,22:00 ~,22:30 ~,23:00 ~,23:30 ~,24:00 ~,24:30 ~
0,평일,1,상선,서울역,150,8.7%,14.8%,25.2%,28.9%,55.5%,67.7%,88.4%,53.5%,58.4%,37.3%,30.6%,47.8%,36.8%,39.8%,38.4%,39.3%,38.4%,32.4%,33.2%,33.3%,26.5%,28.6%,32.3%,32.9%,29.3%,42.3%,48.9%,37.0%,25.5%,18.7%,18.2%,21.6%,21.1%,20.9%,17.6%,13.2%,10.0%,4.8%,0.0%
1,평일,1,상선,시청,151,7.8%,12.5%,20.6%,24.7%,40.7%,51.7%,64.9%,49.1%,47.8%,36.0%,27.4%,41.8%,37.8%,38.0%,37.1%,37.7%,36.6%,34.0%,33.3%,33.6%,29.4%,29.9%,33.7%,35.6%,32.2%,51.2%,55.4%,45.9%,28.8%,22.2%,21.9%,24.5%,27.5%,25.7%,22.1%,16.2%,12.2%,9.1%,1.6%
2,평일,1,상선,종각,152,6.3%,9.7%,15.9%,17.8%,26.1%,34.3%,39.9%,35.4%,37.8%,32.1%,24.9%,38.5%,34.0%,37.1%,35.0%,35.9%,37.5%,34.9%,35.2%,37.6%,32.6%,32.8%,36.1%,41.2%,37.4%,65.6%,67.8%,54.2%,33.5%,30.1%,28.6%,35.9%,39.8%,41.2%,32.8%,23.1%,16.1%,11.0%,2.0%
3,평일,1,상선,종로3가,153,6.3%,9.0%,16.1%,17.7%,27.5%,37.5%,46.0%,36.9%,35.4%,33.2%,27.8%,35.2%,36.6%,40.3%,39.5%,39.5%,42.0%,38.8%,39.6%,42.3%,40.0%,36.8%,42.9%,46.9%,43.7%,68.4%,75.6%,62.7%,38.8%,33.9%,32.3%,39.3%,45.2%,46.3%,39.5%,24.8%,18.3%,12.5%,2.3%
4,평일,1,상선,종로5가,154,5.8%,7.6%,12.7%,14.6%,22.0%,29.1%,33.0%,27.4%,29.5%,27.7%,23.9%,30.7%,32.0%,34.1%,37.2%,36.3%,38.4%,37.4%,38.5%,40.9%,40.2%,35.9%,43.9%,48.8%,46.1%,73.6%,80.2%,65.6%,41.7%,36.4%,33.4%,42.8%,42.8%,53.5%,39.0%,25.6%,18.7%,12.7%,2.2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
values=list(df.columns[5:])
id=[item for item in df.columns if item not in values]
df=pd.melt(df,id_vars=id,value_vars=values)
df

,사용일,호선,구분,역명,역번,variable,value
0,평일,1,상선,서울역,150,05:30 ~,8.7%
1,평일,1,상선,시청,151,05:30 ~,7.8%
2,평일,1,상선,종각,152,05:30 ~,6.3%
3,평일,1,상선,종로3가,153,05:30 ~,6.3%
4,평일,1,상선,종로5가,154,05:30 ~,5.8%
...,...,...,...,...,...,...,...
29050,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN
29051,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN
29052,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN
29053,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN


In [0]:
df = df.rename({'variable':'시간','value':'혼잡도'},axis = 'columns')
df

,사용일,호선,구분,역명,역번,시간,혼잡도
0,평일,1,상선,서울역,150,05:30 ~,8.7%
1,평일,1,상선,시청,151,05:30 ~,7.8%
2,평일,1,상선,종각,152,05:30 ~,6.3%
3,평일,1,상선,종로3가,153,05:30 ~,6.3%
4,평일,1,상선,종로5가,154,05:30 ~,5.8%
...,...,...,...,...,...,...,...
29050,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN
29051,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN
29052,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN
29053,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN


In [0]:
df['time'] = df['시간'].str[:2]
df

,사용일,호선,구분,역명,역번,시간,혼잡도,time
0,평일,1,상선,서울역,150,05:30 ~,8.7%,05
1,평일,1,상선,시청,151,05:30 ~,7.8%,05
2,평일,1,상선,종각,152,05:30 ~,6.3%,05
3,평일,1,상선,종로3가,153,05:30 ~,6.3%,05
4,평일,1,상선,종로5가,154,05:30 ~,5.8%,05
...,...,...,...,...,...,...,...,...
29050,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN,24
29051,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN,24
29052,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN,24
29053,NaN,NaN,NaN,NaN,NaN,24:30 ~,NaN,24


In [0]:
df = df.dropna(axis = 0)
df

,사용일,호선,구분,역명,역번,시간,혼잡도,time
0,평일,1,상선,서울역,150,05:30 ~,8.7%,05
1,평일,1,상선,시청,151,05:30 ~,7.8%,05
2,평일,1,상선,종각,152,05:30 ~,6.3%,05
3,평일,1,상선,종로3가,153,05:30 ~,6.3%,05
4,평일,1,상선,종로5가,154,05:30 ~,5.8%,05
...,...,...,...,...,...,...,...,...
28861,평일,신정지선,외선,도림천,247,24:30 ~,4.0%,24
28862,평일,신정지선,외선,양천구청,248,24:30 ~,3.7%,24
28863,평일,신정지선,외선,신정네거리,249,24:30 ~,4.0%,24
28864,평일,신정지선,외선,까치산,260,24:30 ~,0.0%,24


In [0]:
df = df[(df['호선']=='1')|(df['호선']=='2')|(df['호선']=='3')|(df['호선']=='4')]
df

,사용일,호선,구분,역명,역번,시간,혼잡도,time
0,평일,1,상선,서울역,150,05:30 ~,8.7%,05
1,평일,1,상선,시청,151,05:30 ~,7.8%,05
2,평일,1,상선,종각,152,05:30 ~,6.3%,05
3,평일,1,상선,종로3가,153,05:30 ~,6.3%,05
4,평일,1,상선,종로5가,154,05:30 ~,5.8%,05
...,...,...,...,...,...,...,...,...
28531,평일,4,하선,이촌,430,24:30 ~,6.4%,24
28532,평일,4,하선,동작,431,24:30 ~,6.5%,24
28533,평일,4,하선,총신대입구,432,24:30 ~,5.4%,24
28534,평일,4,하선,사당,433,24:30 ~,5.6%,24


In [0]:
df['상하선'] = df['구분'].replace(['상선','내선','하선','외선'],['1','1','2','2'])
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,사용일,호선,구분,역명,역번,시간,혼잡도,time,상하선
0,평일,1,상선,서울역,150,05:30 ~,8.7%,05,1
1,평일,1,상선,시청,151,05:30 ~,7.8%,05,1
2,평일,1,상선,종각,152,05:30 ~,6.3%,05,1
3,평일,1,상선,종로3가,153,05:30 ~,6.3%,05,1
4,평일,1,상선,종로5가,154,05:30 ~,5.8%,05,1
...,...,...,...,...,...,...,...,...,...
28531,평일,4,하선,이촌,430,24:30 ~,6.4%,24,2
28532,평일,4,하선,동작,431,24:30 ~,6.5%,24,2
28533,평일,4,하선,총신대입구,432,24:30 ~,5.4%,24,2
28534,평일,4,하선,사당,433,24:30 ~,5.6%,24,2


In [0]:
df['sta_no'] = df['역번'].astype(int)
df['상하선'] = df['상하선'].astype(int)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,사용일,호선,구분,역명,역번,시간,혼잡도,time,상하선,sta_no
0,평일,1,상선,서울역,150,05:30 ~,8.7%,05,1,150
1,평일,1,상선,시청,151,05:30 ~,7.8%,05,1,151
2,평일,1,상선,종각,152,05:30 ~,6.3%,05,1,152
3,평일,1,상선,종로3가,153,05:30 ~,6.3%,05,1,153
4,평일,1,상선,종로5가,154,05:30 ~,5.8%,05,1,154
...,...,...,...,...,...,...,...,...,...,...
28531,평일,4,하선,이촌,430,24:30 ~,6.4%,24,2,430
28532,평일,4,하선,동작,431,24:30 ~,6.5%,24,2,431
28533,평일,4,하선,총신대입구,432,24:30 ~,5.4%,24,2,432
28534,평일,4,하선,사당,433,24:30 ~,5.6%,24,2,433


In [0]:
df['congest'] = df['혼잡도'].replace('\%','',regex=True).astype(float) / 100
df['count'] = df['congest'] * 10 * 160
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,사용일,호선,구분,역명,역번,시간,혼잡도,time,상하선,sta_no,congest,count
0,평일,1,상선,서울역,150,05:30 ~,8.7%,05,1,150,0.087,139.2
1,평일,1,상선,시청,151,05:30 ~,7.8%,05,1,151,0.078,124.8
2,평일,1,상선,종각,152,05:30 ~,6.3%,05,1,152,0.063,100.8
3,평일,1,상선,종로3가,153,05:30 ~,6.3%,05,1,153,0.063,100.8
4,평일,1,상선,종로5가,154,05:30 ~,5.8%,05,1,154,0.058,92.8
...,...,...,...,...,...,...,...,...,...,...,...,...
28531,평일,4,하선,이촌,430,24:30 ~,6.4%,24,2,430,0.064,102.4
28532,평일,4,하선,동작,431,24:30 ~,6.5%,24,2,431,0.065,104.0
28533,평일,4,하선,총신대입구,432,24:30 ~,5.4%,24,2,432,0.054,86.4
28534,평일,4,하선,사당,433,24:30 ~,5.6%,24,2,433,0.056,89.6


In [0]:
df_group = df.groupby([df["사용일"], df["호선"], df["역명"], df["sta_no"], df["상하선"], df["time"]], as_index =False)["count"].mean()
df_group

,사용일,호선,역명,sta_no,상하선,time,count
0,평일,1,동대문,155,1,05,136.0
1,평일,1,동대문,155,1,06,161.6
2,평일,1,동대문,155,1,07,262.4
3,평일,1,동대문,155,1,08,460.0
4,평일,1,동대문,155,1,09,416.8
...,...,...,...,...,...,...,...
4515,평일,4,회현,425,2,20,578.4
4516,평일,4,회현,425,2,21,667.2
4517,평일,4,회현,425,2,22,784.0
4518,평일,4,회현,425,2,23,528.8


In [0]:
df_group.to_csv(make_path("차량내혼잡도_2017.csv"), encoding='cp949', index=False)

### 법정동 코드 붙이기(delayed)


In [0]:
# 승하차인원 
metro_file_path = make_path("metro/")
metro_df = read_files(metro_file_path)
metro_df = metro_df.loc[metro_df['사용월']=='201910']
metro_df['지하철역'] = metro_df.지하철역.str.split('(').str[0].str.strip()
metro_df['호선명'] = metro_df.호선명.str.split('호선').str[0].str.strip()
#metro_df.info() # 120 rows

In [0]:
# 법정동코드 
gugun_file_path = make_path("법정동코드_전체자료.txt")
gugun_df = pd.read_csv(gugun_file_path, delimiter = '\t', encoding='CP949', dtype=str,
                       usecols = [0,1])
gugun_df = gugun_df.loc[gugun_df['법정동명'].str.contains('서울')]
#gugun_df.info()

In [0]:
### 유동인구 + 법정동 코드 
# 법정동 코드 구 코드로 만들기
gugun_df['군구'] = gugun_df.법정동명.str.split(' ').str[1]
gugun_df['동이름'] = gugun_df.법정동명.str.split(' ').str[2]
gu_df = gugun_df.loc[ gugun_df['동이름'].isnull() & gugun_df['군구'].notnull() ]
gu_df = gu_df[['법정동코드', '군구']]
#gu_df.info()

In [0]:
# 유동인구에 법정동 코드 붙이기 
flt_df = pd.merge(flt_df, gu_df, on='군구', how='left')
flt_df.info()

In [0]:
# 승하차인원에 역주소 붙이기
r = pd.merge(metro_df, addr_df, on='지하철역', how='left')
r.info()

In [0]:
# 승하차인원에 법정동코드 붙이기
metro_df = pd.merge(r, gu_df, on='군구', how='left')
metro_df.info()

In [0]:
# 승하차인원에 역코드 붙이기(최종 승하차인원 f)
metro_df['호선명'] = metro_df['호선명'].fillna(-1).astype('int')
f = pd.merge(metro_df, stn_cd_df, on=['지하철역', '호선명'], how='left')
f.head()